Détection d’Occupation d’une Salle par Meta-Learning

1. Introduction du Notebook

Dans ce notebook, nous présentons l’ensemble du travail réalisé pour construire un système de détection d’occupation d’une salle à partir de capteurs environnementaux. L’objectif n’était pas seulement d’entraîner un modèle, mais d’aller un peu plus loin avec une approche de meta-learning, où plusieurs classifieurs coopèrent et un meta-modèle décide lequel utiliser.

Le notebook suit un fil logique : exploration des données → entraînement des modèles → génération des méta-features → meta-modèle → pipeline complet.

2. Importation des Librairies

Ici on rassemble les librairies nécessaires pour tout le projet : gestion de données (pandas, numpy), visualisation, modèles scikit-learn, sauvegarde, etc.
Cette cellule sert surtout à centraliser les dépendances.

3. Chargement du Dataset

Dans cette partie, on charge le dataset principal (celui contenant température, humidité, lumière, CO₂, humidity ratio, et étiquette d’occupation).
on vérifie aussi si les colonnes sont cohérentes, et afficher quelques premières lignes pour prendre la température du fichier.

4. Analyse Exploratoire Rapide (EDA)

on regarde ici :

les statistiques générales

la distribution de la variable Occupancy

les valeurs extrêmes possibles

la corrélation entre les variables

Quelques graphiques sont produits pour se faire une idée : histogrammes, heatmap, etc.
Cette partie permet de comprendre si certains capteurs influencent plus que d'autres la présence d’occupants.

5. Prétraitement des Données

Dans cette section, on effectue :

un nettoyage

normalisation des features avec StandardScaler

séparation du dataset en trois ensembles : train, validation, test

L’objectif est de préparer une base propre pour les modèles.

6. Entraînement des Quatre Modèles de Base

Cette partie est consacrée aux quatre modèles qu'on utilise dans le système :

Decision Tree

Random Forest

SVM avec estimation des probabilités

Naive Bayes

Pour chacun, on entraîne le modèle, calcule les métriques habituelles (Accuracy, F1-score) et observe leurs forces et faiblesses.

Les modèles sont ensuite sauvegardés au format .pkl

7. Construction des Métafeatures

Ici, on construis les méta-données qui servent au meta-modèle.
Pour chaque instance du validation set, on récupère :

la confiance maximale du modèle

la différence entre les deux meilleures probabilités (margin)

on fais cela pour les quatre classifieurs

8. Construction du Meta-Dataset

Dans cette section, on décide quel modèle de base “avait raison” pour chaque instance de validation.
Cela nous permet de créer le vecteur meta_y.

Cas particuliers gérés ici :

plusieurs modèles corrects → application de la stratégie fallback

aucun modèle correct → exclusion ou règle alternative

À la fin, on obtiens :

meta_X (méta-features)

meta_y (labels = modèle à choisir)

9. Entraînement du Meta-Modèle (KNN)

on utilise un KNN (k = 5) comme meta-modèle.

Cette partie contient :

entraînement du KNN

validation rapide

sauvegarde du meta-modèle meta_knn.pkl

10. Pipeline Complet d’Inférence

C’est une des parties les plus importantes du notebook.

Ici on construit une fonction complète capable de :

recevoir une nouvelle mesure → X

appliquer le scaler

faire prédire les 4 modèles de base

créer les méta-features

appliquer le KNN pour choisir un modèle

retourner la prédiction finale (occupé / non occupé)

Cette fonction est ensuite sauvegardée et exportée pour l’application Flask.

11. Évaluation Finale du Système

Dans cette section, on teste le pipeline complet sur le test set pour voir son comportement en situation réelle.
Les résultats affichés :

Accuracy finale

F1-score global

Matrice de confusion

Quelques prédictions 


12. Export et Préparation Flask

Ici, on exporte :

scaler

les quatre modèles

meta-knn

la fonction du pipeline final

on assure aussi qu’il est possible de recharger tout ce matériel et prédire en dehors du notebook.